# Advent of Code - Day 12 Year 2024

In [1]:
DAY = 12
YEAR = 2024
#######
import json
import sys 
sys.path.append('../../')
from utils import get_pb_data, rephrase_pb, open_link, render_json
link, data, exmp, ae1, ae2, exmps, out = get_pb_data(DAY, YEAR)

print(f'problem link: {link}')

nb examples: 1
problem link: https://adventofcode.com/2024/day/12


In [3]:
# with open("./_puzzle_data.json", "r") as f: inp = json.loads(f.read())
# render_json(inp)

In [4]:
await rephrase_pb(DAY, YEAR, 1)

**Problem**: Calculate the total cost of fencing garden regions where plants of the same type form a region when connected horizontally or vertically. 

**Cost calculation**:
- Region's price = Area × Perimeter
- Area = Number of plots in the region
- Perimeter = Number of plot sides not touching another plot of same type
- Total price = Sum of all regions' prices

**Key rules**:
- Each plot is a square with 4 sides
- Regions can be separated or nested
- Same plant type can form multiple distinct regions
- Only horizontal and vertical connections count (not diagonal)

**Input Example**:
```
AAAA
BBCD
BBCC
EEEC
```

In this example:
- Region A: area=4, perimeter=10, price=40
- Region B: area=4, perimeter=8, price=32
- Region C: area=4, perimeter=10, price=40
- Region D: area=1, perimeter=4, price=4
- Region E: area=3, perimeter=8, price=24
- Total price = 140

A plot can only belong to a single region

Each plot must belong to a region

I can parse the grid one plot at a time, and determine to which region they belong

Once I have determined every region on the grid and the plots that compose it, I can determine its area and perimeter.

How do I "build" each region ?

How do I model a region ? 

In [6]:
region = {
    "<anchor>": {"type": "R", "plots": set()}
}
# anchor is the coordinate of one of the region's plots
# each plot must belong to only 1 region, so this should work

In [10]:
print(exmp)

AAAA
BBCD
BBCC
EEEC


In [15]:
dat = exmp 
grid = [[c for c in l] for l in dat.splitlines()]
grid

[['A', 'A', 'A', 'A'],
 ['B', 'B', 'C', 'D'],
 ['B', 'B', 'C', 'C'],
 ['E', 'E', 'E', 'C']]

In [16]:
def _format(dat: str) -> list[list[str]]:
    return [[c for c in l] for l in dat.splitlines()]

In [37]:
dat = exmp
grid = _format(dat)
grid

[['A', 'A', 'A', 'A'],
 ['B', 'B', 'C', 'D'],
 ['B', 'B', 'C', 'C'],
 ['E', 'E', 'E', 'C']]

In [38]:
x,y = (0,0)
t = grid[y][x]
print(t)
plots = set()
plots.add((x,y))
regions = {(x,y): {"type": t, "plots": plots}}
regions

A


{(0, 0): {'type': 'A', 'plots': {(0, 0)}}}

In [39]:
deltas = [(1, 0), (-1, 0), (0, 1), (0, -1)]
deltas

[(1, 0), (-1, 0), (0, 1), (0, -1)]

In [40]:
w, h = len(grid[0]), len(grid)
[(x+xd, y+yd) for xd, yd in deltas]

[(1, 0), (-1, 0), (0, 1), (0, -1)]

In [41]:
[(x+xd, y+yd) for xd, yd in deltas if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h]

[(1, 0), (0, 1)]

In [42]:
[(x+xd, y+yd) for xd, yd in deltas if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h and grid[y+yd][x+xd] == t]

[(1, 0)]

In [45]:
[(x+xd, y+yd) for xd, yd in deltas if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h and grid[y+yd][x+xd] == t and (x+xd, y+yd) not in plots]

[(1, 0)]

In [48]:
new_p = [(x+xd, y+yd) for xd, yd in deltas if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h and grid[y+yd][x+xd] == t and (x+xd, y+yd) not in plots]
for p in new_p:
    plots.add(p)  
plots 

{(0, 0), (1, 0)}

In [56]:
def _region(x, y, regions):
    regs = [v["plots"] for v in regions.values() if (x,y) in v["plots"]]
    if len(regs) == 0:
        return None
    if len(regs) > 1:
        raise ValueError('len should be <= 1') 
    return regs[0]

In [55]:
regions = {}
x,y = (0,0)
for j in range(h):
    for i in range(w):
        print((x+i, y+j))

(0, 0)
(1, 0)
(2, 0)
(3, 0)
(0, 1)
(1, 1)
(2, 1)
(3, 1)
(0, 2)
(1, 2)
(2, 2)
(3, 2)
(0, 3)
(1, 3)
(2, 3)
(3, 3)


In [98]:
def _region(x, y, regions):
    regs = [v["plots"] for v in regions.values() if (x,y) in v["plots"]]
    if len(regs) == 0:
        return None
    if len(regs) > 1:
        raise ValueError(f'len should be <= 1: ({x}, {y})') 
    return regs[0]

In [66]:
dat = exmp
grid = _format(dat)
regions = {}
for j in range(h):
    for i in range(w):
        x, y = i,j
        print(x,y)
        region = _region(x, y, regions)
        t = grid[y][x]
        if region is None:
            print('region is none')
            regions[(x,y)] = {"type": t, "plots": {(x,y)}}
        else:
            neighbours = [(x+xd, y+yd) for xd, yd in deltas if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h and grid[y+yd][x+xd] == t]
            for n in neighbours:
                region.add(n)

0 0
region is none
0 1
region is none
0 2
region is none
0 3
region is none


In [70]:
dat = exmp
grid = _format(dat)
regions = {}

In [79]:
i, j = 2,1
x, y = i,j
print(x,y)
region = _region(x, y, regions)
t = grid[y][x]
if region is None:
    print('region is none')
    regions[(x,y)] = {"type": t, "plots": {(x,y)}}
    region = regions[(x,y)]["plots"]
neighbours = [(x+xd, y+yd) for xd, yd in deltas if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h and grid[y+yd][x+xd] == t]
for n in neighbours:
    region.add(n)
regions

2 1
region is none


{(0, 0): {'type': 'A', 'plots': {(0, 0), (1, 0), (2, 0), (3, 0)}},
 (0, 1): {'type': 'B', 'plots': {(0, 1), (0, 2), (1, 1), (1, 2)}},
 (0, 3): {'type': 'E', 'plots': {(0, 3), (1, 3)}},
 (2, 1): {'type': 'C', 'plots': {(2, 1), (2, 2)}}}

In [81]:
dat = exmp
grid = _format(dat)
regions = {}
for j in range(h):
    for i in range(w):
        x, y = i,j
        print(x,y)
        region = _region(x, y, regions)
        t = grid[y][x]
        if region is None:
            print('region is none')
            regions[(x,y)] = {"type": t, "plots": {(x,y)}}
            region = regions[(x,y)]["plots"]
        neighbours = [(x+xd, y+yd) for xd, yd in deltas if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h and grid[y+yd][x+xd] == t]
        for n in neighbours:
            region.add(n)
regions

0 0
region is none
1 0
2 0
3 0
0 1
region is none
1 1
2 1
region is none
3 1
region is none
0 2
1 2
2 2
3 2
0 3
region is none
1 3
2 3
3 3


{(0, 0): {'type': 'A', 'plots': {(0, 0), (1, 0), (2, 0), (3, 0)}},
 (0, 1): {'type': 'B', 'plots': {(0, 1), (0, 2), (1, 1), (1, 2)}},
 (2, 1): {'type': 'C', 'plots': {(2, 1), (2, 2), (3, 2), (3, 3)}},
 (3, 1): {'type': 'D', 'plots': {(3, 1)}},
 (0, 3): {'type': 'E', 'plots': {(0, 3), (1, 3), (2, 3)}}}

In [91]:
def build_region(grid) -> dict:
    regions = {}
    w, h = len(grid[0]), len(grid)
    for j in range(h):
        for i in range(w):
            x, y = i,j
            t = grid[y][x]
            region = _region(x, y, regions)
            if region is None:
                regions[(x,y)] = {"type": t, "plots": {(x,y)}}
                region = regions[(x,y)]["plots"]
            neighbours = [(x+xd, y+yd) for xd, yd in deltas if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h and grid[y+yd][x+xd] == t]
            for n in neighbours: region.add(n)
    return regions
    

In [93]:
dat = exmp
grid = _format(dat)
print(grid)
build_region(grid)

[['A', 'A', 'A', 'A'], ['B', 'B', 'C', 'D'], ['B', 'B', 'C', 'C'], ['E', 'E', 'E', 'C']]


{(0, 0): {'type': 'A', 'plots': {(0, 0), (1, 0), (2, 0), (3, 0)}},
 (0, 1): {'type': 'B', 'plots': {(0, 1), (0, 2), (1, 1), (1, 2)}},
 (2, 1): {'type': 'C', 'plots': {(2, 1), (2, 2), (3, 2), (3, 3)}},
 (3, 1): {'type': 'D', 'plots': {(3, 1)}},
 (0, 3): {'type': 'E', 'plots': {(0, 3), (1, 3), (2, 3)}}}

In [94]:
ex2 = """OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
"""

In [95]:
dat = ex2
grid = _format(dat)
print(grid)
build_region(grid)

[['O', 'O', 'O', 'O', 'O'], ['O', 'X', 'O', 'X', 'O'], ['O', 'O', 'O', 'O', 'O'], ['O', 'X', 'O', 'X', 'O'], ['O', 'O', 'O', 'O', 'O']]


{(0, 0): {'type': 'O',
  'plots': {(0, 0),
   (0, 1),
   (0, 2),
   (0, 3),
   (0, 4),
   (1, 0),
   (1, 2),
   (1, 4),
   (2, 0),
   (2, 1),
   (2, 2),
   (2, 3),
   (2, 4),
   (3, 0),
   (3, 2),
   (3, 4),
   (4, 0),
   (4, 1),
   (4, 2),
   (4, 3),
   (4, 4)}},
 (1, 1): {'type': 'X', 'plots': {(1, 1)}},
 (3, 1): {'type': 'X', 'plots': {(3, 1)}},
 (1, 3): {'type': 'X', 'plots': {(1, 3)}},
 (3, 3): {'type': 'X', 'plots': {(3, 3)}}}

In [96]:
ex3 = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
"""

In [99]:
dat = ex3
grid = _format(dat)
print(grid)
build_region(grid)

[['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'F', 'F'], ['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'C', 'F'], ['V', 'V', 'R', 'R', 'R', 'C', 'C', 'F', 'F', 'F'], ['V', 'V', 'R', 'C', 'C', 'C', 'J', 'F', 'F', 'F'], ['V', 'V', 'V', 'V', 'C', 'J', 'J', 'C', 'F', 'E'], ['V', 'V', 'I', 'V', 'C', 'C', 'J', 'J', 'E', 'E'], ['V', 'V', 'I', 'I', 'I', 'C', 'J', 'J', 'E', 'E'], ['M', 'I', 'I', 'I', 'I', 'I', 'J', 'J', 'E', 'E'], ['M', 'I', 'I', 'I', 'S', 'I', 'J', 'E', 'E', 'E'], ['M', 'M', 'M', 'I', 'S', 'S', 'J', 'E', 'E', 'E']]


ValueError: len should be <= 1: (6, 2)

In [102]:
grid

[['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'F', 'F'],
 ['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'C', 'F'],
 ['V', 'V', 'R', 'R', 'R', 'C', 'C', 'F', 'F', 'F'],
 ['V', 'V', 'R', 'C', 'C', 'C', 'J', 'F', 'F', 'F'],
 ['V', 'V', 'V', 'V', 'C', 'J', 'J', 'C', 'F', 'E'],
 ['V', 'V', 'I', 'V', 'C', 'C', 'J', 'J', 'E', 'E'],
 ['V', 'V', 'I', 'I', 'I', 'C', 'J', 'J', 'E', 'E'],
 ['M', 'I', 'I', 'I', 'I', 'I', 'J', 'J', 'E', 'E'],
 ['M', 'I', 'I', 'I', 'S', 'I', 'J', 'E', 'E', 'E'],
 ['M', 'M', 'M', 'I', 'S', 'S', 'J', 'E', 'E', 'E']]

I can't build regions by simply iterating over each element. I need to "follow the thread" until all contiguous regions are found

In [103]:
def is_in_existing_region(x,y, regions):
    regs = [v["plots"] for v in regions.values() if (x,y) in v["plots"]]
    assert len(regs) in [0,1], "regions length should be 0 or 1"
    return len(regs) == 1

In [117]:
def build_region_2(x,y, grid, regions):
    t = grid[y][x]
    w, h = len(grid[0]), len(grid)
    if not is_in_existing_region(x,y, regions):
        regions[(x,y)] = {'type': t, "plots": {(x,y)}}
    plots = [v["plots"] for _,v in regions.items() if (x,y) in v["plots"]][0]
    new_nbg = [
        (x+xd, y+yd) 
        for xd, yd in deltas 
        if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h 
        and grid[y+yd][x+xd] == t 
        and (x+xd, y+yd) not in plots
    ]
    for n in new_nbg: plots.add(n)
    for xn, yn in new_nbg: build_region_2(xn, yn, grid, regions)
    return regions


In [118]:
dat = ex3
grid = _format(dat)
regions = {}
x, y = 0,0
build_region_2(x, y, grid, regions)

{(0, 0): {'type': 'R',
  'plots': {(0, 0),
   (0, 1),
   (1, 0),
   (1, 1),
   (2, 0),
   (2, 1),
   (2, 2),
   (2, 3),
   (3, 0),
   (3, 1),
   (3, 2),
   (4, 2)}}}

In [119]:
def _format(dat: str) -> list[list[str]]:
    grid = [[c for c in l] for l in dat.splitlines()]
    w, h = len(grid[0]), len(grid)
    return grid, w, h

In [126]:
dat = ex2
grid, w, h = _format(dat)
print(w, h)
regions = {}
for j in range(h):
    for i in range(w):
        build_region_2(i, j, grid, regions)
len(regions)

5 5


5

In [128]:
def build_regions(dat: str):
    grid, w, h = _format(dat)
    regions = {}
    for j in range(h):
        for i in range(w):
            build_region_2(i, j, grid, regions)
    return regions

In [134]:
dat = exmp
build_regions(dat)

{(0, 0): {'type': 'A', 'plots': {(0, 0), (1, 0), (2, 0), (3, 0)}},
 (0, 1): {'type': 'B', 'plots': {(0, 1), (0, 2), (1, 1), (1, 2)}},
 (2, 1): {'type': 'C', 'plots': {(2, 1), (2, 2), (3, 2), (3, 3)}},
 (3, 1): {'type': 'D', 'plots': {(3, 1)}},
 (0, 3): {'type': 'E', 'plots': {(0, 3), (1, 3), (2, 3)}}}

difficulty left: calculate number of sides

In [165]:
def build_region_3(x,y, grid, regions, processed):
    
    if (x,y) in processed:
        return 
    processed.add((x,y))

    t = grid[y][x]
    w, h = len(grid[0]), len(grid)
    if not is_in_existing_region(x,y, regions):
        regions[(x,y)] = {'type': t, "plots": {(x,y)}, "edges": []}

    region = [regions[k] for k,v in regions.items() if (x,y) in v['plots']][0]
    plots = region['plots']
    edges = region['edges']
    n_nbr = [
        (x+xd, y+yd) 
        for xd, yd in deltas 
        if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h 
        and grid[y+yd][x+xd] == t 
        and (x+xd, y+yd) not in plots
    ]
    n_edg = [
        (x+xd, y+yd) 
        for xd, yd in deltas 
        if x+xd == -1 or x+xd == w or y+yd == -1 or y+yd == h 
        or grid[y+yd][x+xd] != t 
    ]
    for (xe, ye) in n_edg: edges.append((xe,ye))
    for (xn, yn) in n_nbr: plots.add((xn, yn))
    for (xn, yn) in n_nbr: build_region_3(xn, yn, grid, regions, processed)


In [166]:
def build_regions(dat: str):
    grid, w, h = _format(dat)
    regions = {}
    processed = set()
    for j in range(h):
        for i in range(w):
            build_region_3(i, j, grid, regions, processed)
    return regions

In [169]:
dat = ex2
regions = build_regions(dat)
regions

{(0, 0): {'type': 'O',
  'plots': {(0, 0),
   (0, 1),
   (0, 2),
   (0, 3),
   (0, 4),
   (1, 0),
   (1, 2),
   (1, 4),
   (2, 0),
   (2, 1),
   (2, 2),
   (2, 3),
   (2, 4),
   (3, 0),
   (3, 2),
   (3, 4),
   (4, 0),
   (4, 1),
   (4, 2),
   (4, 3),
   (4, 4)},
  'edges': [(-1, 0),
   (0, -1),
   (1, 1),
   (1, -1),
   (2, -1),
   (3, 1),
   (3, -1),
   (5, 0),
   (4, -1),
   (5, 1),
   (3, 1),
   (5, 2),
   (3, 3),
   (3, 1),
   (1, 3),
   (1, 1),
   (-1, 2),
   (1, 3),
   (-1, 3),
   (-1, 4),
   (0, 5),
   (1, 5),
   (1, 3),
   (2, 5),
   (3, 5),
   (3, 3),
   (5, 4),
   (4, 5),
   (3, 3),
   (1, 3),
   (5, 3),
   (3, 3),
   (3, 1),
   (1, 1),
   (1, 1),
   (-1, 1)]},
 (1, 1): {'type': 'X',
  'plots': {(1, 1)},
  'edges': [(2, 1), (0, 1), (1, 2), (1, 0)]},
 (3, 1): {'type': 'X',
  'plots': {(3, 1)},
  'edges': [(4, 1), (2, 1), (3, 2), (3, 0)]},
 (1, 3): {'type': 'X',
  'plots': {(1, 3)},
  'edges': [(2, 3), (0, 3), (1, 4), (1, 2)]},
 (3, 3): {'type': 'X',
  'plots': {(3, 3)},
  'ed

In [174]:
def _region_cost(region):
    area = len(region["plots"])
    perimeter = len(region["edges"])
    return area * perimeter

In [179]:
def sol1(dat: str):
    regions = build_regions(dat)
    return sum([_region_cost(regions[r]) for r in regions])


In [180]:
dat = exmp 
sol1(dat)

140

In [181]:
dat = ex2 
sol1(dat)

772

In [182]:
dat = ex3 
sol1(dat)

1930

In [183]:
dat = data 
sol1(dat)

1549354

## part 2

In [ ]:
await rephrase_pb(DAY, YEAR, 2)

from edges I should be able to deduce sides

In [184]:
dat = exmp 
regions = build_regions(dat)

In [190]:
edg = set(regions[(2,1)]['edges'])
edg

{(1, 1), (1, 2), (2, 0), (2, 3), (3, 1), (3, 4), (4, 2), (4, 3)}

In [192]:
edg

{(1, 1), (1, 2), (2, 0), (2, 3), (3, 1), (3, 4), (4, 2), (4, 3)}

In [194]:
side_symb = {
    (1, 0): '>',
    (-1, 0): '<',
    (0,1): 'v',
    (0, -1):'^',
}

In [209]:
def build_region_4(x,y, grid, regions, processed):
    
    if (x,y) in processed:
        return 
    processed.add((x,y))

    t = grid[y][x]
    w, h = len(grid[0]), len(grid)
    if not is_in_existing_region(x,y, regions):
        regions[(x,y)] = {'type': t, "plots": {(x,y)}, "edges": []}

    region = [regions[k] for k,v in regions.items() if (x,y) in v['plots']][0]
    plots = region['plots']
    edges = region['edges']
    n_nbr = [
        (x+xd, y+yd) 
        for xd, yd in deltas 
        if x+xd >= 0 and x+xd < w and y+yd >= 0 and y+yd < h 
        and grid[y+yd][x+xd] == t 
        and (x+xd, y+yd) not in plots
    ]
    n_edg = [
        (x+xd, y+yd, side_symb[(xd,yd)]) 
        for xd, yd in deltas 
        if x+xd == -1 or x+xd == w or y+yd == -1 or y+yd == h 
        or grid[y+yd][x+xd] != t 
    ]
    for (xe, ye, s) in n_edg: edges.append((xe,ye,s))
    for (xn, yn) in n_nbr: plots.add((xn, yn))
    for (xn, yn) in n_nbr: build_region_4(xn, yn, grid, regions, processed)


In [210]:
def build_regions_sol2(dat: str):
    grid, w, h = _format(dat)
    regions = {}
    processed = set()
    for j in range(h):
        for i in range(w):
            build_region_4(i, j, grid, regions, processed)
    return regions

In [211]:
dat = exmp 
regions = build_regions_sol2(dat)

In [212]:
def sol2(dat: str):
    regions = build_regions_sol2(dat)
    return sum([_region_cost(regions[r]) for r in regions])


In [215]:
region = regions[(2,1)]
region

{'type': 'C',
 'plots': {(2, 1), (2, 2), (3, 2), (3, 3)},
 'edges': [(3, 1, '>'),
  (1, 1, '<'),
  (2, 0, '^'),
  (1, 2, '<'),
  (2, 3, 'v'),
  (4, 2, '>'),
  (3, 1, '^'),
  (4, 3, '>'),
  (2, 3, '<'),
  (3, 4, 'v')]}

In [219]:
edges = region['edges']
edges

[(3, 1, '>'),
 (1, 1, '<'),
 (2, 0, '^'),
 (1, 2, '<'),
 (2, 3, 'v'),
 (4, 2, '>'),
 (3, 1, '^'),
 (4, 3, '>'),
 (2, 3, '<'),
 (3, 4, 'v')]

In [220]:
e = edges[0]
e

(3, 1, '>')

In [227]:
used = set()
sides = []

In [230]:
f = [z for z in edges if z[2] == e[2] and z[0] == e[0] and z not in used]
if len(f) > 0:
    sides.append(f)
    for z in f: used.add(z)

In [232]:
e = edges[1]

In [234]:
f = [z for z in edges if z[2] == e[2] and z[0] == e[0] and z not in used]
if len(f) > 0:
    sides.append(f)
    for z in f: used.add(z)

In [235]:
sides

[[(3, 1, '>')], [(1, 1, '<'), (1, 2, '<')]]

In [236]:
e = edges[2]
e

(2, 0, '^')

In [237]:
fix_idx_map = {'^': 1, 'v': 1, '>': 0, '<':0}

In [240]:
e[2]

'^'

In [241]:
f = [z for z in edges if z[2] == e[2] and z[fix_idx_map[e[2]]] == e[fix_idx_map[e[2]]] and z not in used]
if len(f) > 0:
    sides.append(f)
    for z in f: used.add(z)

In [244]:
sides

[[(3, 1, '>')], [(1, 1, '<'), (1, 2, '<')], [(2, 0, '^')]]

In [245]:
e = edges[3]
f = [z for z in edges if z[2] == e[2] and z[fix_idx_map[e[2]]] == e[fix_idx_map[e[2]]] and z not in used]
if len(f) > 0:
    sides.append(f)
    for z in f: used.add(z)

In [248]:
dat = exmp 
regions = build_regions_sol2(dat)
region = regions[(0,0)]
edges = region['edges']
used = set()
sides = []
for e in edges:
    f = [z for z in edges if z[2] == e[2] and z[fix_idx_map[e[2]]] == e[fix_idx_map[e[2]]] and z not in used]
    if len(f) > 0:
        sides.append(f)
        for z in f: used.add(z)
sides

[[(-1, 0, '<')],
 [(0, 1, 'v'), (1, 1, 'v'), (2, 1, 'v'), (3, 1, 'v')],
 [(0, -1, '^'), (1, -1, '^'), (2, -1, '^'), (3, -1, '^')],
 [(4, 0, '>')]]

In [250]:
def _sides(region):
    edges = region['edges']
    used = set()
    sides = []
    for e in edges:
        f = [z for z in edges if z[2] == e[2] and z[fix_idx_map[e[2]]] == e[fix_idx_map[e[2]]] and z not in used]
        if len(f) > 0:
            sides.append(f)
            for z in f: used.add(z)
    return sides

In [253]:
print(ex2)

OOOOO
OXOXO
OOOOO
OXOXO
OOOOO



In [254]:
dat = ex2
regions = build_regions_sol2(dat)
region = regions[(0,0)]
sorted(_sides(region), key=len)

[[(1, 1, 'v'), (3, 1, 'v')],
 [(3, 1, '<'), (3, 3, '<')],
 [(3, 3, 'v'), (1, 3, 'v')],
 [(3, 1, '^'), (1, 1, '^')],
 [(1, 3, '>'), (1, 1, '>')],
 [(1, 3, '^'), (3, 3, '^')],
 [(3, 3, '>'), (3, 1, '>')],
 [(1, 3, '<'), (1, 1, '<')],
 [(-1, 0, '<'), (-1, 2, '<'), (-1, 3, '<'), (-1, 4, '<'), (-1, 1, '<')],
 [(0, -1, '^'), (1, -1, '^'), (2, -1, '^'), (3, -1, '^'), (4, -1, '^')],
 [(5, 0, '>'), (5, 1, '>'), (5, 2, '>'), (5, 4, '>'), (5, 3, '>')],
 [(0, 5, 'v'), (1, 5, 'v'), (2, 5, 'v'), (3, 5, 'v'), (4, 5, 'v')]]

In [255]:
def _sides(region):
    edges = region['edges']
    used = set()
    sides = []
    for e in edges:
        f = [z for z in edges if z[2] == e[2] and z[fix_idx_map[e[2]]] == e[fix_idx_map[e[2]]] and z not in used]
        if len(f) > 0:
            sides.append(f)
            for z in f: used.add(z)
    return sides

In [257]:
def _contiguous_edges(e, edges):
    ...

In [259]:
e

(3, -1, '^')

In [260]:
edges

[(-1, 0, '<'),
 (0, 1, 'v'),
 (0, -1, '^'),
 (1, 1, 'v'),
 (1, -1, '^'),
 (2, 1, 'v'),
 (2, -1, '^'),
 (4, 0, '>'),
 (3, 1, 'v'),
 (3, -1, '^')]

In [261]:
sides = {}
for e in edges:
    fix_idx = fix_idx_map[e[2]]
    mov_idx = 1 if fix_idx == 0 else 0
    ss1 = [
        v for k,v in sides.items() 
        if k[2] == e[2] 
        and k[fix_idx] == e[fix_idx] 
        and any([(z[mov_idx] == e[mov_idx] + 1) or (z[mov_idx] == e[mov_idx] - 1) for z in v])
    ]
    if len(ss1) == 0:
        sides[e] = [e]
    else:
        ss1[0].append(e)

In [262]:
sides

{(-1, 0, '<'): [(-1, 0, '<')],
 (0, 1, 'v'): [(0, 1, 'v'), (1, 1, 'v'), (2, 1, 'v'), (3, 1, 'v')],
 (0, -1, '^'): [(0, -1, '^'), (1, -1, '^'), (2, -1, '^'), (3, -1, '^')],
 (4, 0, '>'): [(4, 0, '>')]}

In [270]:
dat = ex2
regions = build_regions_sol2(dat)
region = regions[(0,0)]
edges = region['edges']

In [283]:
sides = {}
for e in edges:
    print(e)
    print('@')
    fix_idx = fix_idx_map[e[2]]
    mov_idx = 1 if fix_idx == 0 else 0
    ss1 = [
        v for k,v in sides.items() 
        if k[2] == e[2] 
        and k[fix_idx] == e[fix_idx] 
        and any([(z[mov_idx] == e[mov_idx] + 1) or (z[mov_idx] == e[mov_idx] - 1) for z in v])
    ]
    print(f'ss1: {ss1}')
    if len(ss1) == 0:
        sides[e] = [e]
    elif len(ss1) > 1:
        assert False, 'should be < 1'
    else:
        ss1[0].append(e)
    
    for sss in sides:
        print(f'{sss}: {sides[sss]}')
    print('---')
len(sides)

(-1, 0, '<')
@
ss1: []
(-1, 0, '<'): [(-1, 0, '<')]
---
(0, -1, '^')
@
ss1: []
(-1, 0, '<'): [(-1, 0, '<')]
(0, -1, '^'): [(0, -1, '^')]
---
(1, 1, 'v')
@
ss1: []
(-1, 0, '<'): [(-1, 0, '<')]
(0, -1, '^'): [(0, -1, '^')]
(1, 1, 'v'): [(1, 1, 'v')]
---
(1, -1, '^')
@
ss1: [[(0, -1, '^')]]
(-1, 0, '<'): [(-1, 0, '<')]
(0, -1, '^'): [(0, -1, '^'), (1, -1, '^')]
(1, 1, 'v'): [(1, 1, 'v')]
---
(2, -1, '^')
@
ss1: [[(0, -1, '^'), (1, -1, '^')]]
(-1, 0, '<'): [(-1, 0, '<')]
(0, -1, '^'): [(0, -1, '^'), (1, -1, '^'), (2, -1, '^')]
(1, 1, 'v'): [(1, 1, 'v')]
---
(3, 1, 'v')
@
ss1: []
(-1, 0, '<'): [(-1, 0, '<')]
(0, -1, '^'): [(0, -1, '^'), (1, -1, '^'), (2, -1, '^')]
(1, 1, 'v'): [(1, 1, 'v')]
(3, 1, 'v'): [(3, 1, 'v')]
---
(3, -1, '^')
@
ss1: [[(0, -1, '^'), (1, -1, '^'), (2, -1, '^')]]
(-1, 0, '<'): [(-1, 0, '<')]
(0, -1, '^'): [(0, -1, '^'), (1, -1, '^'), (2, -1, '^'), (3, -1, '^')]
(1, 1, 'v'): [(1, 1, 'v')]
(3, 1, 'v'): [(3, 1, 'v')]
---
(5, 0, '>')
@
ss1: []
(-1, 0, '<'): [(-1, 0, '<')]


AssertionError: should be < 1

This approach doesn't work

In [291]:
dat = ex2
regions = build_regions_sol2(dat)
region = regions[(0,0)]
edges = region['edges']

In [292]:
already_used = set()
sides = {}

for e in edges:
    if e in already_used:
        continue
    already_used.add(e)
    sides[e] = [e]
    fix_idx = fix_idx_map[e[2]]
    mov_idx = 1 if fix_idx == 0 else 0
    i = -1
    while True:
        lz = [z for z in edges if z[2] == e[2] and z[fix_idx] == e[fix_idx] and z[mov_idx] == e[mov_idx] + i]
        if len(lz) == 0:
            break
        elif len(lz) > 1:
            assert False, "impossible"
        z = lz[0]
        sides[e].append(z)
        already_used.add(z)
        i -= 1
    i = +1
    while True:
        lz = [z for z in edges if z[2] == e[2] and z[fix_idx] == e[fix_idx] and z[mov_idx] == e[mov_idx] + i]
        if len(lz) == 0:
            break
        elif len(lz) > 1:
            assert False, "impossible"
        z = lz[0]
        sides[e].append(z)
        already_used.add(z)
        i += 1
print(len(sides))
sides

20


{(-1, 0, '<'): [(-1, 0, '<'),
  (-1, 1, '<'),
  (-1, 2, '<'),
  (-1, 3, '<'),
  (-1, 4, '<')],
 (0, -1, '^'): [(0, -1, '^'),
  (1, -1, '^'),
  (2, -1, '^'),
  (3, -1, '^'),
  (4, -1, '^')],
 (1, 1, 'v'): [(1, 1, 'v')],
 (3, 1, 'v'): [(3, 1, 'v')],
 (5, 0, '>'): [(5, 0, '>'),
  (5, 1, '>'),
  (5, 2, '>'),
  (5, 3, '>'),
  (5, 4, '>')],
 (3, 1, '<'): [(3, 1, '<')],
 (3, 3, 'v'): [(3, 3, 'v')],
 (3, 1, '^'): [(3, 1, '^')],
 (1, 3, 'v'): [(1, 3, 'v')],
 (1, 1, '^'): [(1, 1, '^')],
 (1, 3, '>'): [(1, 3, '>')],
 (0, 5, 'v'): [(0, 5, 'v'),
  (1, 5, 'v'),
  (2, 5, 'v'),
  (3, 5, 'v'),
  (4, 5, 'v')],
 (1, 3, '^'): [(1, 3, '^')],
 (3, 3, '^'): [(3, 3, '^')],
 (3, 3, '>'): [(3, 3, '>')],
 (1, 3, '<'): [(1, 3, '<')],
 (3, 3, '<'): [(3, 3, '<')],
 (3, 1, '>'): [(3, 1, '>')],
 (1, 1, '<'): [(1, 1, '<')],
 (1, 1, '>'): [(1, 1, '>')]}

In [293]:
def _get_sides(edges):

    already_used = set()
    sides = {}

    for e in edges:
        if e in already_used:
            continue
        already_used.add(e)
        sides[e] = [e]
        fix_idx = fix_idx_map[e[2]]
        mov_idx = 1 if fix_idx == 0 else 0
        i = -1
        while True:
            lz = [z for z in edges if z[2] == e[2] and z[fix_idx] == e[fix_idx] and z[mov_idx] == e[mov_idx] + i]
            if len(lz) == 0:
                break
            elif len(lz) > 1:
                assert False, "impossible"
            z = lz[0]
            sides[e].append(z)
            already_used.add(z)
            i -= 1
        i = +1
        while True:
            lz = [z for z in edges if z[2] == e[2] and z[fix_idx] == e[fix_idx] and z[mov_idx] == e[mov_idx] + i]
            if len(lz) == 0:
                break
            elif len(lz) > 1:
                assert False, "impossible"
            z = lz[0]
            sides[e].append(z)
            already_used.add(z)
            i += 1
    return sides

In [298]:
dat = ex2
regions = build_regions_sol2(dat)
region = regions[(1,3)]
edges = region['edges']
len(_get_sides(edges))

4

In [299]:
def _region_cost(region):
    area = len(region["plots"])
    edges = region["edges"]
    sides = _get_sides(edges)
    perimeter = len(sides)
    return area * perimeter

In [300]:
def sol2(dat: str):
    regions = build_regions_sol2(dat)
    return sum([_region_cost(regions[r]) for r in regions])


In [302]:
dat = ex2
sol2(dat)

436

In [303]:
ex4 = """EEEEE
EXXXX
EEEEE
EXXXX
EEEEE
"""

In [304]:
dat = ex4
sol2(dat)

236

In [305]:
ex5 = """AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA
"""

In [306]:
dat = ex5
sol2(dat)

368

In [307]:
dat = ex3
sol2(dat)

1206

In [308]:
dat = data
sol2(dat)

937032

## thoughts

- these grid-type problems don't focus on computational complexity, but require breaking down the problem well

- my solution feels messy: for each edge, move on one side until no contiguous edge found, then same in the opposite direction

- There probably is a better way to model the problem

- It requires modeling the FRONTIER between grid cases

- It also requires taking into account indexes outside of the grid